In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pymongo
import dns
import datetime
from time import sleep
import random
import json

from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

# Project defined dependencies
import config
from pprint import pprint
import cl_parser


In [4]:
# Create the craiglist parser
my_parser = cl_parser.Parser() 

In [5]:
for listing in my_parser.listings_collection.find():
    print(listing['data_id'])
    break


7278244211


In [7]:
my_parser.listings_collection.find().count()    

9069

In [9]:
find_result = my_parser.listings_collection.find_one({'data_id': '7280736862'})
pprint(find_result)

{'_id': ObjectId('60330e42566e11a3b7aedc54'),
 'created_datetime': datetime.datetime(2021, 2, 22, 1, 52, 2, 943000),
 'data_id': '7280736862',
 'listing_addrcountry': 'US',
 'listing_address': '2051 Geneva St',
 'listing_addrlocality': 'Vista',
 'listing_addrregion': 'CA',
 'listing_addrstreet': '740 Paseo Buena Vista',
 'listing_addrzip': '92085',
 'listing_attributes': ['EV charging',
                        'cats are OK - purrr',
                        'dogs are OK - wooof',
                        'apartment',
                        'w/d in unit',
                        'no smoking',
                        'detached garage'],
 'listing_availability': 'available now',
 'listing_bath': 1,
 'listing_bed': '1',
 'listing_bedbath': '1BR / 1Ba',
 'listing_datetime': '2021-02-21 11:42',
 'listing_latitude': '33.208926',
 'listing_longitude': '-117.234635',
 'listing_petsallowed': 'True',
 'listing_price': '$1,995',
 'listing_smokingallowed': 'False',
 'listing_sqft': '732ft2',
 'listi

In [26]:
def insert_listing(cl_result_row):
    data_id = result.h3.a['data-id']
    listing_title = result.find('a', class_='result-title').text
    listing_price = result.a.span.text
    listing_url = result.a['href']
    listing_datetime = result.time['datetime']
    created_datetime = datetime.datetime.utcnow()
    modified_datetime = datetime.datetime.utcnow()
        
    # Print results
    print('-------------')
    print(data_id)
    print(listing_title)
    print(listing_price)
    print(listing_url)
    print(listing_datetime)        
    print(created_datetime)


    # Dictionary to be inserted as a MongoDB document
    post = {
        'data_id': data_id,
        'created_datetime': created_datetime,
        'listing_title': listing_title,
        'listing_price': listing_price,
        'listing_url': listing_url,
        'listing_date': listing_datetime
        
    }

    if listings_collection.find({'data_id': data_id}).count() == 0:
        print("Insert new listing! ")
        # Run only if all fields are available
        if (listing_title and listing_price and listing_url and listing_datetime and data_id and created_datetime):
            listings_collection.insert_one(post)
            
    else: 
        print(f"Listing {data_id} already exists, update existing listing!")

        doc = listings_collection.find_one_and_update(
            {'data_id' : data_id},
            {'$set':
                {
                    'modified_datetime': modified_datetime,
                    'listing_title': listing_title,                
                    'listing_price': listing_price,
                    'listing_url': listing_url,
                    'listing_datetime': listing_datetime
                }
                
            },upsert=True
        )



In [529]:
# saved page one of results locally for temp solution to build out scripts/functions
# filepath = os.path.join('..','resources','cl_sandiego_northcounty_apa.html')
# with open(filepath, encoding='utf-8') as file:
#     html = file.read()

# soup = BeautifulSoup(html)

In [7]:
for listing in listings_collection.find():
    print(listing)
    break

{'_id': ObjectId('60330e35566e11a3b7aedbf6'), 'data_id': '7278244211', 'created_datetime': datetime.datetime(2021, 2, 22, 1, 51, 49, 230000), 'listing_title': 'Dog Park, In-Unit Laundry, Extra storage', 'listing_price': '$2,095', 'listing_url': 'https://sandiego.craigslist.org/nsd/apa/d/carlsbad-dog-park-in-unit-laundry-extra/7278244211.html', 'listing_date': '2021-02-21 15:13', 'listing_addrcountry': 'US', 'listing_address': '3402 Calle Odessa', 'listing_addrlocality': 'Carlsbad', 'listing_addrregion': 'CA', 'listing_addrstreet': '3402 Calle Odessa', 'listing_addrzip': '92009', 'listing_attributes': ['cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'no smoking', 'carport'], 'listing_availability': '', 'listing_bath': 1, 'listing_bed': '1', 'listing_bedbath': '1BR / 1Ba', 'listing_latitude': '33.077861', 'listing_longitude': '-117.234826', 'listing_petsallowed': 'True', 'listing_smokingallowed': 'False', 'listing_type': 'Apartment', 'listing_sqft': '679ft2'}


In [8]:
listings_collection.find().count()    

7951

In [9]:
def get_listing_details(listing_url):
    # URL of page to be scraped
    url = listing_url

    sleep(random.randint(1,3))
    # Retrieve page with the requests module
    response = requests.get(url)
    # Create BeautifulSoup object; parse with 'lxml'
    soup = BeautifulSoup(response.text,'lxml')    
    return soup

In [15]:
def insert_listing_details(cl_result_details,data_id):   
    # Examine the results, then determine element that contains sought info
    # results are returned as an iterable list
    result_details = cl_result_details.find_all('div', class_='mapAndAttrs')
    
    viewposting = cl_result_details.find_all('div', class_='viewposting')
    listing_latitude = viewposting[0]['data-latitude']
    listing_longitude = viewposting[0]['data-longitude']
    
    # mapaddress = cl_result_details.find_all('div', class_='mapaddress')
    # listing_address = mapaddress[0].text
    
    attrgroups = cl_result_details.find_all('p', class_='attrgroup')
    
    listing_availability = ''
    listing_sqft = ''

    for attrgroup in attrgroups:
        listing_attributes = []
        attrspan = attrgroup.find_all('span')
        for span in attrspan:
            if ((span.text.lower().find('br') != -1) & (span.text.lower().find('ba') != -1)):
                listing_bedbath = span.text                
            elif span.text.lower().find('ft2') != -1:
                listing_sqft = span.text
            elif span.text.lower().find('available') != -1:
                listing_availability = span.text
            else: 
                listing_attributes.append(span.text)

    listing_type = ''
    listing_bed = ''
    listing_bath = ''
    listing_petsallowed = ''
    listing_smokingallowed = ''

    listing_addrcountry = ''
    listing_addrlocality = ''
    listing_addrregion = ''
    listing_addrregion = ''
    listing_addrzip = ''
    listing_addrstreet = ''
        
    soup_scripts = cl_result_details.find_all('script',id='ld_posting_data')

    # Getting dictionary
    scripts_dict = json.loads(soup_scripts[0].contents[0].strip())

    # Pretty Printing JSON string back
    # print(json.dumps(scripts_dict, indent = 4, sort_keys=True))
        
    if '@type' in scripts_dict:
        listing_type = scripts_dict['@type']
    if 'numberOfBedrooms' in scripts_dict:
        listing_bed = scripts_dict['numberOfBedrooms']
    if 'numberOfBathroomsTotal' in scripts_dict:
        listing_bath = scripts_dict['numberOfBathroomsTotal']
    if 'petsAllowed' in scripts_dict:
        listing_petsallowed = str(scripts_dict['petsAllowed'])
    if 'smokingAllowed' in scripts_dict:
        listing_smokingallowed = str(scripts_dict['smokingAllowed'])
    

    if 'address' in scripts_dict:
        address_dict = scripts_dict['address']
        if 'addressCountry' in address_dict:
            listing_addrcountry = address_dict['addressCountry']
        if 'addressLocality' in address_dict:
            listing_addrlocality = address_dict['addressLocality']
        if 'addressRegion' in address_dict:
            listing_addrregion = address_dict['addressRegion']
        if 'postalCode' in address_dict:
            listing_addrzip = address_dict['postalCode']
        if 'streetAddress' in address_dict:
            listing_addrstreet = address_dict['streetAddress']
        

    print(data_id)
    print(listing_latitude)
    print(listing_longitude)    
    print(listing_bedbath)
    print(listing_sqft)
    print(listing_availability)
    print(listing_attributes)
    print(listing_addrcountry)
    print(listing_addrlocality)
    print(listing_addrregion)
    print(listing_addrzip)
    print(listing_addrstreet)
    print(listing_type)
    print(listing_bed)
    print(listing_bath)
    print(listing_petsallowed)
    print(listing_smokingallowed)
    print('------------------------------')

    doc = listings_collection.find_one_and_update(
        {'data_id' : data_id},
        {'$set':
            {
                'listing_latitude': listing_latitude,
                'listing_longitude': listing_longitude,                
                'listing_bedbath':listing_bedbath,
                'listing_sqft':listing_sqft,
                'listing_availability':listing_availability,
                'listing_attributes':listing_attributes,
                'listing_addrcountry':listing_addrcountry,
                'listing_addrlocality':listing_addrlocality,
                'listing_addrregion':listing_addrregion,
                'listing_addrzip':listing_addrzip,
                'listing_addrstreet':listing_addrstreet,
                'listing_type':listing_type,
                'listing_bed':listing_bed,
                'listing_bath':listing_bath,
                'listing_petsallowed':listing_petsallowed,
                'listing_smokingallowed':listing_smokingallowed                
            }
            
        },upsert=True
    )
    





In [11]:
find_result = listings_collection.find_one({'data_id': '7280736862'})
print(find_result)

{'_id': ObjectId('60330e42566e11a3b7aedc54'), 'data_id': '7280736862', 'created_datetime': datetime.datetime(2021, 2, 22, 1, 52, 2, 943000), 'listing_title': 'Strike While the Iron is Hot! 6-Weeks FREE!', 'listing_price': '$1,995', 'listing_url': 'https://sandiego.craigslist.org/nsd/apa/d/vista-strike-while-the-iron-is-hot/7280736862.html', 'listing_date': '2021-02-21 11:42', 'listing_address': '2051 Geneva St', 'listing_attributes': ['EV charging', 'cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'no smoking', 'detached garage'], 'listing_availability': 'available now', 'listing_bedbath': '1BR / 1Ba', 'listing_latitude': '33.208926', 'listing_longitude': '-117.234635', 'listing_addrcountry': 'US', 'listing_addrlocality': 'Vista', 'listing_addrregion': 'CA', 'listing_addrstreet': '740 Paseo Buena Vista', 'listing_addrzip': '92085', 'listing_bath': 1, 'listing_bed': '1', 'listing_petsallowed': 'True', 'listing_smokingallowed': 'False', 'listing_type': 'Apartment

In [27]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324


[WDM] - Driver [C:\Users\melis\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache


In [28]:
# listings results
url_listings_northsd = 'https://sandiego.craigslist.org/search/nsd/apa'
url_listings_eastsd = 'https://sandiego.craigslist.org/search/esd/apa'
url_lisings_cityofsd = 'https://sandiego.craigslist.org/search/csd/apa'
url_lisings_southsd = 'https://sandiego.craigslist.org/search/ssd/apa'

urls = []
urls.append(url_listings_northsd)
urls.append(url_listings_eastsd)
urls.append(url_lisings_cityofsd)
urls.append(url_lisings_southsd)



In [39]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324


[WDM] - Driver [C:\Users\melis\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache


In [30]:
# iterate through the list of URLs, return each region listing and then run through the pages
for url in urls:
    browser.visit(url)

    # Iterate through all pages
    for x in range(26):
        # HTML object
        html_listings = browser.html
        # Parse HTML with Beautiful Soup
        soup_listings = BeautifulSoup(html_listings, 'html.parser')
        # Retrieve all elements that contain book information
        soup_listings = BeautifulSoup(html_listings)

        results = soup_listings.find_all('li', class_='result-row')

        # Loop through returned results
        for result in results:
            try:
                insert_listing(result)                                                                  
            except Exception as e:
                print(e)        
        try:
            browser.links.find_by_partial_text('next').click()          
        except:
            print("Scraping Complete")
            break


----------
7277983892
NEW CABINETS, FLOORING AND MORE!!!
$1,350
https://sandiego.craigslist.org/ssd/apa/d/chula-vista-new-cabinets-flooring-and/7277983892.html
2021-02-15 14:56
2021-02-22 09:46:33.308858
Listing 7277983892 already exists, update existing listing!
-------------
7277983695
NEW CABINETS, FLOORING AND MORE!!!
$1,350
https://sandiego.craigslist.org/ssd/apa/d/chula-vista-new-cabinets-flooring-and/7277983695.html
2021-02-15 14:56
2021-02-22 09:46:33.461424
Listing 7277983695 already exists, update existing listing!
-------------
7277983589
NEW CABINETS, FLOORING AND MORE!!!
$1,350
https://sandiego.craigslist.org/ssd/apa/d/chula-vista-new-cabinets-flooring-and/7277983589.html
2021-02-15 14:56
2021-02-22 09:46:33.620827
Listing 7277983589 already exists, update existing listing!
-------------
7277983463
NEW CABINETS, FLOORING AND MORE!!!
$1,350
https://sandiego.craigslist.org/ssd/apa/d/chula-vista-new-cabinets-flooring-and/7277983463.html
2021-02-15 14:56
2021-02-22 09:46:33.78

Scraping Complete


In [ ]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [38]:
x = 0

for l in listings_collection.find():
    
    isNeedUpdate = False
    try: 
        listing_address = l['listing_addrzip']
    except:
        isNeedUpdate = True

    try: 
        listing_address = l['listing_sqft']
    except:
        isNeedUpdate = True

    """try: 
        listing_addrstreet = l['listing_addrstreet']
        if (listing_addrstreet == '1510 S. MELROSE'):
            isNeedUpdate = True  
    except:
        isNeedUpdate = True        
    """
    if isNeedUpdate == True:
        data_id = l['data_id']
        url_details = l['listing_url']
        
        print('Index ' + str(x) + ' Updating listing details data_id ' + data_id)

        if (url_details != ''):
            sleep(random.randint(1,3))
            browser.visit(url_details)
            html_details = browser.html

            soup_details = BeautifulSoup(html_details)
            
            try:
                insert_listing_details(soup_details, data_id)
            except Exception as e:               
                print(e)  
            
    else:
        listing_address = ''
    
    x += 1
    if x > 7952:            
        print('Scrape Completed')
        break


treet parking']
US
National City
CA
91951
1627 E 9TH ST
Apartment
2
1


------------------------------
Index 7885 Updating listing details data_id 7277797361
7277797361
32.679613
-117.089485
3BR / 1Ba
870ft2

['application fee details: $35 per applicant (18 years and over)', 'apartment', 'laundry on site', 'street parking']
US
National City
CA
91951
1627 E 9TH ST
Apartment
3
1


------------------------------
Index 7886 Updating listing details data_id 7277795512
7277795512
32.617667
-117.041529
2BR / 1.5Ba

available now
['cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'laundry on site', 'off-street parking']
US
Chula Vista
CA
91911

Apartment
2
2
True

------------------------------
Index 7887 Updating listing details data_id 7277793060
7277793060
33.192833
-117.279512
2BR / 1Ba
890ft2
available now
['apartment', 'w/d in unit', 'carport']
US
Oceanside
CA
92056
333 Emerald Dr
Apartment
2
1


------------------------------
Index 7888 Updating listing details data_id 72777919